# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-10.57,76,42,6.12,GS,1719273901
1,1,yuzhno-kurilsk,44.0306,145.8556,10.81,99,100,3.23,RU,1719273845
2,2,smara,26.7384,-11.6719,19.68,78,0,5.30,EH,1719273903
3,3,greenwood,34.1668,-82.1165,32.93,41,14,2.99,US,1719273905
4,4,adamstown,-25.0660,-130.1015,22.70,87,56,6.64,PN,1719273906


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

#  Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriImagery",
    size="Humidity",
    frame_width=800,
    frame_height=600,
    scale=0.5,
    color="City"
)

# Display the map
map_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
min_temp_s = city_data_df["Max Temp"] > 20
max_temp_s = city_data_df["Max Temp"] < 30
wind_speed_s = city_data_df["Wind Speed"] > 10


In [17]:
print(sum(min_temp_s))
print(sum(max_temp_s))
print(sum(wind_speed_s))

306
531
8


In [18]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp_df = city_data_df[min_temp_s & max_temp_s & wind_speed_s]

# Drop any rows with null values
clean_city_temp_df = city_temp_df.dropna()

#Display sample data
clean_city_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,port mathurin,-19.6833,63.4167,23.29,75,19,11.02,MU,1719273963
88,88,atafu village,-8.5421,-172.5159,28.29,74,95,10.64,TK,1719273892
244,244,trincomalee,8.5711,81.2335,27.98,75,100,10.61,LK,1719274247
277,277,kavaratti,10.5669,72.6420,27.61,85,100,12.10,IN,1719274232
384,384,bandarbeyla,9.4942,50.8122,27.64,61,76,12.79,SO,1719274448


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,grytviken,GS,-10.57,-54.2811,-36.5092,
1,yuzhno-kurilsk,RU,10.81,44.0306,145.8556,
2,smara,EH,19.68,26.7384,-11.6719,
3,greenwood,US,32.93,34.1668,-82.1165,
4,adamstown,PN,22.70,-25.0660,-130.1015,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 1000
limit = 1
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"]
    lat = hotel_df.loc[index, "Lat"]
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)
    print(name_address.url)

    # Convert the API response to JSON format
    name_address = name_address.json()
   

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=5eff84bee723431a85004c36a139a606&filter=circle%3A-36.5092%2C-54.2811%2C1000&bias=proximity%3A-36.5092%2C-54.2811
grytviken - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=5eff84bee723431a85004c36a139a606&filter=circle%3A145.8556%2C44.0306%2C1000&bias=proximity%3A145.8556%2C44.0306
yuzhno-kurilsk - nearest hotel: Айсберг
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=5eff84bee723431a85004c36a139a606&filter=circle%3A-11.6719%2C26.7384%2C1000&bias=proximity%3A-11.6719%2C26.7384
smara - nearest hotel: Hôtel Amine فندق الأمين
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=5eff84bee723431a85004c36a139a606&filter=circle%3A-82.1165%2C34.1668%2C1000&bias=proximity%3A-82.1165%2C34.1668
greenwood - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?ca

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,grytviken,GS,-10.57,-54.2811,-36.5092,No hotel found
1,yuzhno-kurilsk,RU,10.81,44.0306,145.8556,Айсберг
2,smara,EH,19.68,26.7384,-11.6719,Hôtel Amine فندق الأمين
3,greenwood,US,32.93,34.1668,-82.1165,No hotel found
4,adamstown,PN,22.70,-25.0660,-130.1015,No hotel found
...,...,...,...,...,...,...
558,sirte,LY,26.15,31.2089,16.5887,فندق المدينة-سرت
559,ujae,MH,26.47,8.9322,165.7642,No hotel found
560,saint ann's bay,JM,27.89,18.4358,-77.2010,No hotel found
561,otofuke,JP,17.83,42.9917,143.2003,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    color = "City"
)

# Display the map
hotel_map_plot